In [2]:
import pandas as pd
import camelot # um Tabellen im PDFs zu erkennen
import PyPDF2 # um Text aus PDFs zu lesen
import re

In [3]:
file = "Sessionsprotokolle/SR_5115_2206.pdf"

In [8]:
dokument = PyPDF2.PdfReader(file)

In [20]:
inhalt = dokument.pages[576].extract_text()
datenliste = inhalt.split("\n")

In [21]:
datenliste

['Geschäft / Objet: ',
 '22.3153 Ordnungsantrag',
 "Motion d'ordre",
 'Gegenstand / Objet du vote: Abstimmung über den Ordnungsantrag Zanetti zu 22.3153',
 'Abstimmung vom / Vote du: 13.06.2022 16:38:17',
 'Bauer Philippe - NE Knecht Hansjörg - AG',
 'Baume-Schneider Elisabeth 0 JU Kuprecht Alex - SZ',
 'Bischof Pirmin 0 SO Maret Marianne + VS',
 'Burkart Thierry - AG Mazzone Lisa 0 GE',
 'Carobbio Guscetti Marina + TI Michel Matthias - ZG',
 'Caroni Andrea - AR Minder Thomas - SH',
 'Chassot Isabelle + FR Müller Damian 0 LU',
 'Chiesa Marco - TI Noser Ruedi - ZH',
 'Dittli Josef 0 UR Rechsteiner Paul + SG',
 'Engler Stefan + GR Reichmuth Othmar - SZ',
 'Ettlin Erich + OW Rieder Beat + VS',
 'Fässler Daniel = AI Salzmann Werner - BE',
 'Français Olivier + VD Schmid Martin - GR',
 'Gapany Johanna 0 FR Sommaruga Carlo + GE',
 'Germann Hannes - SH Stark Jakob 0 TG',
 'Gmür-',
 'SchönenbergerAndrea 0 LU Stöckli Hans + BE',
 'Graf Maya + BL Thorens Goumaz Adèle + VD',
 'Häberli-Koller Brigi

In [28]:
def first_substring(strings, substring):
    return next(i for i, string in enumerate(strings) if substring in string)

first_substring(datenliste, 'Abstimmung vom / Vote du:')

4

In [11]:
regex_gnummer = r"(\d\d\.\S+)"
regex_datum = r"\d\d\.\d\d\.\d\d\d\d \d\d\:\d\d\:\d\d"

## !!! Fehlerkontrolle manuell !!!

- Seite 200: Daten von 18.071 fehlen, das erste Geschäft > Tabelle wird nicht erkannt - warum?

- Seite 212: Daten von 20.008-2 fehlen; 
    - es muss was damit zu tun haben, dass davor eine leere Seite im Doc ist
    - Erkenntnis: er findet die Tabelle eine Seite früher, auf S. 211! (Warum auch immmer...)

In [6]:
dokument.pages[212].extract_text()

'8. Juni 2022 427 Ständerat 20.069\nArt. 11\nProposition de la majorité\n...\nb. ... avec un ﬁlm ou un jeu vidéo;\nc. ... une charge disproportionnée pour l\'organisation de bran-\nche ou les acteurs ...\ncbis. Biffer\n...\nh. ... de la réglementation par l\'organisation de branche con-\ncernée, en ...\ni. ... aux acteurs membres de l\'organisation de branche con-\ncernée en ...\n...\nProposition de la minorité\n(Baume-Schneider, Carobbio Guscetti, Graf Maya, Rechstei-\nner Paul)\nLet. b\nSelon Conseil fédéral, mais:\n... aux acteurs membres de l\'organisation de branche con-\ncernée en ...\nPräsident (Hefti Thomas, Präsident): Wir behandeln diese\nBestimmungen gemeinsam.\nMichel Matthias (RL, ZG), für die Kommission: Bei Artikel\n11 geht es um den Inhalt der erwarteten Jugendschutzre-\ngelung. Hier beantragen wir eine Anpassung von Litera b.\nDort soll der letzte Satzteil gestrichen werden, in welchem\nsteht, dass die Jugendschutzregelung "Regeln [...] zum Um-\ngang mit Filmen oder Vi

In [165]:
testtabelle = camelot.read_pdf(file, flavor='stream', pages = str(200), )

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/camelot/parsers/stream.py:453: UserWarning: No tables found on page-200
  warnings.warn(f"No tables found on {base_filename}")


# Lösung:

- Wir holen für solche Fälle die Daten aus dem Text (.extract_text)
- dazu splitten wir den Text in eine Liste (gits schon: die "datenliste"
- um die richtige Position anzusteuern, suchen wir den Index der Leerzeile. (datenliste.index(' '))
- Es sind dann immer die nächste 24 Listenelemente
- Wir müssen jedesmal noch die blöde Gmür-Schöneberger reparieren
- Dann generieren wir eine Liste mit allen Kantonskürzeln, damit wir jeweil nach diesen die beiden Einträge pro Zeile splitten können
- dann splitten wir erneut nach den ersten beiden Leerschlägen links des Kantons und übernehmen die Werte 
- am Schluss der Liste kommt das hier ' '

In [ ]:
else:

In [173]:
tabellenanfang = datenliste.index(' ')+1

In [174]:
tabellenanfang

10

In [274]:
# so holen wir die nächsten 24 Zeilen raus
tabelle = datenliste[tabellenanfang:(tabellenanfang+24)]

In [275]:
tabelle

['Bauer Philippe + NE Kuprecht Alex + SZ',
 'Baume-Schneider Elisabeth + JU Levrat Christian + FR',
 'Bischof Pirmin 0 SO Maret Marianne + VS',
 'Burkart Thierry + AG Mazzone Lisa + GE',
 'Carobbio Guscetti Marina + TI Michel Matthias + ZG',
 'Caroni Andrea 0 AR Minder Thomas + SH',
 'Chiesa Marco 0 TI Müller Damian + LU',
 'Dittli Josef + UR Noser Ruedi + ZH',
 'Engler Stefan + GR Rechsteiner Paul 0 SG',
 'Ettlin Erich 0 OW Reichmuth Othmar + SZ',
 'Fässler Daniel + AI Rieder Beat + VS',
 'Français Olivier + VD Salzmann Werner + BE',
 'Gapany Johanna + FR Schmid Martin + GR',
 'Germann Hannes + SH Sommaruga Carlo 0 GE',
 'Gmür-',
 'SchönenbergerAndrea + LU Stark Jakob + TG',
 'Graf Maya + BL Stöckli Hans P BE',
 'Häberli-Koller Brigitte + TG Thorens Goumaz Adèle + VD',
 'Hefti Thomas + GL Vara Céline 0 NE',
 'Hegglin Peter + ZG Wicki Hans + NW',
 'Herzog Eva + BS Würth Benedikt + SG',
 'Jositsch Daniel + ZH Zanetti Roberto + SO',
 "Juillard Charles + JU Z'graggen Heidi + UR",
 'Knecht

In [276]:
# So flicken wir die Gmür-Schnönenberger:
tabelle.remove("Gmür-")
tabelle = [w.replace('SchönenbergerAndrea', 'Gmür-Schönenberger Andrea') for w in tabelle]

In [277]:
tabelle

['Bauer Philippe + NE Kuprecht Alex + SZ',
 'Baume-Schneider Elisabeth + JU Levrat Christian + FR',
 'Bischof Pirmin 0 SO Maret Marianne + VS',
 'Burkart Thierry + AG Mazzone Lisa + GE',
 'Carobbio Guscetti Marina + TI Michel Matthias + ZG',
 'Caroni Andrea 0 AR Minder Thomas + SH',
 'Chiesa Marco 0 TI Müller Damian + LU',
 'Dittli Josef + UR Noser Ruedi + ZH',
 'Engler Stefan + GR Rechsteiner Paul 0 SG',
 'Ettlin Erich 0 OW Reichmuth Othmar + SZ',
 'Fässler Daniel + AI Rieder Beat + VS',
 'Français Olivier + VD Salzmann Werner + BE',
 'Gapany Johanna + FR Schmid Martin + GR',
 'Germann Hannes + SH Sommaruga Carlo 0 GE',
 'Gmür-Schönenberger Andrea + LU Stark Jakob + TG',
 'Graf Maya + BL Stöckli Hans P BE',
 'Häberli-Koller Brigitte + TG Thorens Goumaz Adèle + VD',
 'Hefti Thomas + GL Vara Céline 0 NE',
 'Hegglin Peter + ZG Wicki Hans + NW',
 'Herzog Eva + BS Würth Benedikt + SG',
 'Jositsch Daniel + ZH Zanetti Roberto + SO',
 "Juillard Charles + JU Z'graggen Heidi + UR",
 'Knecht Han

In [283]:
# und jetzt trennen wir jede Zeile beim ersten Auftauchen eines Kantons (jenes mit Leerschlag)
tabelle_getrennt = []
kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

for paar in tabelle:
    for kanton in kantonsliste:
        trenner = kanton + " "
        if trenner in paar:
            trennung = paar.split(trenner)
            
            # (dann müssen wir den bei wplit verloren gegangenen Kanton, den Trenner, wieder hinzufügen)
            trennung[0] = trennung[0]+kanton
            
            tabelle_getrennt.append(trennung[0])
            tabelle_getrennt.append(trennung[1])
            
tabelle_getrennt
        

['Bauer Philippe + NE',
 'Kuprecht Alex + SZ',
 'Baume-Schneider Elisabeth + JU',
 'Levrat Christian + FR',
 'Bischof Pirmin 0 SO',
 'Maret Marianne + VS',
 'Burkart Thierry + AG',
 'Mazzone Lisa + GE',
 'Carobbio Guscetti Marina + TI',
 'Michel Matthias + ZG',
 'Caroni Andrea 0 AR',
 'Minder Thomas + SH',
 'Chiesa Marco 0 TI',
 'Müller Damian + LU',
 'Dittli Josef + UR',
 'Noser Ruedi + ZH',
 'Engler Stefan + GR',
 'Rechsteiner Paul 0 SG',
 'Ettlin Erich 0 OW',
 'Reichmuth Othmar + SZ',
 'Fässler Daniel + AI',
 'Rieder Beat + VS',
 'Français Olivier + VD',
 'Salzmann Werner + BE',
 'Gapany Johanna + FR',
 'Schmid Martin + GR',
 'Germann Hannes + SH',
 'Sommaruga Carlo 0 GE',
 'Gmür-Schönenberger Andrea + LU',
 'Stark Jakob + TG',
 'Graf Maya + BL',
 'Stöckli Hans P BE',
 'Häberli-Koller Brigitte + TG',
 'Thorens Goumaz Adèle + VD',
 'Hefti Thomas + GL',
 'Vara Céline 0 NE',
 'Hegglin Peter + ZG',
 'Wicki Hans + NW',
 'Herzog Eva + BS',
 'Würth Benedikt + SG',
 'Jositsch Daniel + ZH',


In [ ]:
# diese neue Liste trennen wir jetzt auf, und zwar nach Leerschlägen von hinten, wegen Doppelnamen etc. > mit rsplit!

In [296]:
for eintrag in tabelle_getrennt:
    eintrag_neu = eintrag.rsplit(" ", 2)
    
    

['Bauer Philippe', '+', 'NE']
['Kuprecht Alex', '+', 'SZ']
['Baume-Schneider Elisabeth', '+', 'JU']
['Levrat Christian', '+', 'FR']
['Bischof Pirmin', '0', 'SO']
['Maret Marianne', '+', 'VS']
['Burkart Thierry', '+', 'AG']
['Mazzone Lisa', '+', 'GE']
['Carobbio Guscetti Marina', '+', 'TI']
['Michel Matthias', '+', 'ZG']
['Caroni Andrea', '0', 'AR']
['Minder Thomas', '+', 'SH']
['Chiesa Marco', '0', 'TI']
['Müller Damian', '+', 'LU']
['Dittli Josef', '+', 'UR']
['Noser Ruedi', '+', 'ZH']
['Engler Stefan', '+', 'GR']
['Rechsteiner Paul', '0', 'SG']
['Ettlin Erich', '0', 'OW']
['Reichmuth Othmar', '+', 'SZ']
['Fässler Daniel', '+', 'AI']
['Rieder Beat', '+', 'VS']
['Français Olivier', '+', 'VD']
['Salzmann Werner', '+', 'BE']
['Gapany Johanna', '+', 'FR']
['Schmid Martin', '+', 'GR']
['Germann Hannes', '+', 'SH']
['Sommaruga Carlo', '0', 'GE']
['Gmür-Schönenberger Andrea', '+', 'LU']
['Stark Jakob', '+', 'TG']
['Graf Maya', '+', 'BL']
['Stöckli Hans', 'P', 'BE']
['Häberli-Koller Brigitte'

## Jetzt ergänzen wir den Loop um diese Ausnahmeregel >>> das ist die Lösung!

In [309]:
master_df_list = []

for seite in seitenliste:
    inhalt = dokument.pages[seite].extract_text()
    datenliste = inhalt.split("\n")
    
    for zeile in range(len(datenliste)):
        if "Abstimmung vom" in datenliste[zeile]:
        
            datum = re.findall(regex_datum, datenliste[zeile])[0]
            
            print(datum)
    
    for zeile in range(len(datenliste)):
        if "Geschäft" in datenliste[zeile]:
            geschäftszeile = datenliste[(zeile+1)]
            
            gnummer = (re.findall(regex_gnummer, geschäftszeile))[0]
            geschäft = geschäftszeile.strip(str(gnummer)+" ")
            
            print(gnummer)
            print(geschäft)
            print(" ")
    
            
    tables = camelot.read_pdf(file, flavor='lattice', pages = str(seite))
    
    try: 
        for table in range(0,2):
            abstimmungs_df = tables[table].df
    
            zeilen = len(abstimmungs_df.index)

            for zeile in range(0,zeilen):    
                name = abstimmungs_df.loc[zeile, 1] + " " +abstimmungs_df.loc[zeile, 0]
                kanton = abstimmungs_df.loc[zeile, 3]
                verhalten = abstimmungs_df.loc[zeile, 2]

                mini_dict = {
                    "Datum" : datum,
                    "Nummer" : gnummer,
                    "Geschäft" : geschäft,
                    "Politiker" : name,
                    "Kanton" : kanton,
                    "Verhalten" : verhalten}
    
                master_df_list.append(mini_dict)
        
    #hier kommt der unten entwickelte Alternativweg rein, bei dem Text ausgelesen wird, weil Tabelle nicht erkannt wird:
    except:
        # So holen wir die richtigen 24 Elemente aus der Tabelle
        tabellenanfang = datenliste.index(' ')+1
        tabelle = datenliste[tabellenanfang:(tabellenanfang+24)]
        
        # So flicken wir die Gmür-Schnönenberger:
        tabelle.remove("Gmür-")
        tabelle = [w.replace('SchönenbergerAndrea', 'Gmür-Schönenberger Andrea') for w in tabelle]
        
        # und jetzt trennen wir jede Zeile beim ersten Auftauchen eines Kantons (jenes mit Leerschlag)
        tabelle_getrennt = []
        kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

        for paar in tabelle:
            for kanton in kantonsliste:
                trenner = kanton + " "
                if trenner in paar:
                    trennung = paar.split(trenner)
            
                    # (dann müssen wir den bei wplit verloren gegangenen Kanton, den Trenner, wieder hinzufügen)
                    trennung[0] = trennung[0]+kanton
            
                    tabelle_getrennt.append(trennung[0])
                    tabelle_getrennt.append(trennung[1])
                    
        for eintrag in tabelle_getrennt:
            eintrag_neu = eintrag.rsplit(" ", 2)
            
            name = eintrag_neu[0]
            kanton = eintrag_neu[2]
            verhalten = eintrag_neu[1]
            
            mini_dict = {
            "Datum" : datum,
            "Nummer" : gnummer,
            "Geschäft" : geschäft,
            "Politiker" : name,
            "Kanton" : kanton,
            "Verhalten" : verhalten}
    
            master_df_list.append(mini_dict)

09.03.2020 18:19:01
18.071
Terrorismus und organisierte Kriminalität. Übereinkommen des Europarates
 


/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/camelot/parsers/lattice.py:417: UserWarning: No tables found on page-200
  "No tables found on {}".format(os.path.basename(self.rootname))


04.03.2020 11:37:37
18.076
Bundesgesetz über das Internationale Privatrecht. 12. Kapitel: Internationale Schiedsgerichtsbarkeit
 
02.03.2020 17:45:41
18.092
Erwerbsersatzgesetz. Mutterschaftsentschädigung bei längerem Spitalaufenthalt des Neugeborenen
 
02.03.2020 17:02:00
19.030
Tierseuchengesetz. Änderung
 
09.03.2020 19:40:30
19.032-1
Polizeiliche Massnahmen zur Bekämpfung von Terrorismus. Bundesgesetz
 
03.03.2020 09:39:02
19.033
Einführung des automatischen Informationsaustauschs über Finanzkonten mit weiteren Partnerstaaten ab
 
02.03.2020 17:21:13
19.047
Übereinkommen des Europarats gegen den Handel mit menschlichen Organen. Genehmigung und
 
10.03.2020 10:18:42
19.051-1
Überbrückungsleistung für ältere Arbeitslose. Bundesgesetz
 
04.03.2020 08:33:21
19.471
Pa.Iv. Comte. Opfer fürsorgerischer Zwangsmassnahmen. Fristverlängerung
 
04.03.2020 08:35:27
19.471
Pa.Iv. Comte. Opfer fürsorgerischer Zwangsmassnahmen. Fristverlängerung
 
11.03.2020 09:12:04
20.008-1
Aussenwirtschaftspoli

In [310]:
master_df = pd.DataFrame(master_df_list)
master_df

,Datum,Nummer,Geschäft,Politiker,Kanton,Verhalten
0,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Bauer Philippe,NE,+
1,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Kuprecht Alex,SZ,+
2,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Baume-Schneider Elisabeth,JU,+
3,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Levrat Christian,FR,+
4,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Bischof Pirmin,SO,+
...,...,...,...,...,...,...
639,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Peter Hegglin,ZG,0
640,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Eva Herzog,BS,+
641,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Daniel Jositsch,ZH,+
642,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Charles Juillard,JU,+


In [311]:
master_df['Datum'].value_counts()

09.03.2020 18:19:01    46
04.03.2020 11:37:37    46
02.03.2020 17:45:41    46
02.03.2020 17:02:00    46
09.03.2020 19:40:30    46
03.03.2020 09:39:02    46
02.03.2020 17:21:13    46
10.03.2020 10:18:42    46
04.03.2020 08:33:21    46
04.03.2020 08:35:27    46
11.03.2020 09:12:04    46
11.03.2020 09:12:50    46
11.03.2020 09:13:36    46
11.03.2020 09:14:16    46
Name: Datum, dtype: int64